# First Stage Thrust Curves

The reference I can find with thrust curves for the Lambda vehicle has a total thrust curve for the entire first stage, the main motor and both boosters combined. We would like to know what the indavidual thrusts are.

We can probably make a pretty good guess using the thrust of the first stage motor by itself and then subtracting from the total.

## First Stage Total Thrust

In [ ]:
# dependencies
from numpy import loadtxt, interp, copy
import matplotlib.pyplot as plt
%matplotlib inline

g_0 = 9.80665

LB2KG = 0.453592
IN2M = 0.0254
SF2KGM = 1.35581795

total_thrust_time, total_thrust = loadtxt('../thrust_booster_and_stage1.csv', delimiter=',', unpack=True)
stage1_thrust_time, stage1_thrust = loadtxt('../735-33-thrustcurve.csv', delimiter=',', unpack=True)

In [ ]:
fig, ax1 = plt.subplots(figsize=(18,8))
plt.title(r"L-4S-2 First Stage Total Thrust")
plt.ylabel(r"Thrust [kN]")
plt.xlabel(r"Launch Elapsed Time [s]")

plt.plot(total_thrust_time, (total_thrust*g_0)/1000.0, alpha=0.7, label="L-4S-2 Vehicle Data")

#plt.ylim([0,5])
plt.xlim([0,30])
#ax1.legend(loc=1)
plt.show()

We can see the booster thrust die down after about 7 seconds.

An earlier reference shows a thrust curve from the first stage motor by itself:

In [ ]:
fig, ax1 = plt.subplots(figsize=(18,8))
plt.title(r"L-4S First Stage Thrust")
plt.ylabel(r"Thrust [kN]")
plt.xlabel(r"Launch Elapsed Time [s]")

plt.plot(total_thrust_time, (total_thrust*g_0)/1000.0, alpha=0.7, label="L-4S-2 Vehicle Total First Stage Thrust")
plt.plot(stage1_thrust_time, stage1_thrust/1000.0, alpha=0.7, label="735 Motor Thrust")

#plt.ylim([0,5])
plt.xlim([0,30])
ax1.legend(loc=1)
plt.show()

And then subtract that curve to get just the booster thrust by itself. There are two boosters, so dividing by two will give the thrust of just one by itself.

In [ ]:
# first 8.0 seconds of time
stage1_boost_time = stage1_thrust_time[:80]

# interpolate to get same time base
stage1_boost_thrust = interp(stage1_boost_time, total_thrust_time, total_thrust*g_0)

# subtract the first stage thrust
stage1_boost_thrust = stage1_boost_thrust - stage1_thrust[:80]

# divide by 2 (two boosters)
stage1_boost_thrust = stage1_boost_thrust/2.0

# scale down so that the last point === 0
stage1_boost_thrust = stage1_boost_thrust - stage1_boost_thrust[-1]

# fix blip at beginning
stage1_boost_thrust[0] = stage1_boost_thrust[2]
stage1_boost_thrust[1] = stage1_boost_thrust[2]


fig, ax1 = plt.subplots(figsize=(6,8))
plt.title(r"L-4S First Stage Single Booster Thrust")
plt.ylabel(r"Thrust [kN]")
plt.xlabel(r"Launch Elapsed Time [s]")

plt.plot(stage1_boost_time, stage1_boost_thrust/1000.0, alpha=0.7, label="First Stage Booster")

#plt.ylim([0,200])
plt.xlim([0,8.7])
#ax1.legend(loc=1)
plt.show()

## Reconstructed Thrust Curve

If we put it all back together, it should exactly match our original thrustcurve

In [ ]:
comp_thrust = copy(stage1_thrust)
comp_thrust[:80] = stage1_thrust[:80] + stage1_boost_thrust * 2

fig, ax1 = plt.subplots(figsize=(18,8))
plt.title(r"L-4S-2 First Composite Thrust")
plt.ylabel(r"Thrust [kN]")
plt.xlabel(r"Launch Elapsed Time [s]")

plt.plot(total_thrust_time, (total_thrust*g_0)/1000.0, 'k-', lw=3.0, alpha=0.5, label="L-4S-2 Vehicle Data")
plt.plot(stage1_thrust_time, comp_thrust/1000.0, alpha=1.0, label="Composite First Stage Thrust (Total reconstructed thrust curve)")
plt.plot(stage1_thrust_time, stage1_thrust/1000.0, 'b--', lw=1.0, label="First Stage Main Motor")
plt.plot(stage1_boost_time, stage1_boost_thrust/1000.0, 'g--', lw=1.0, label="First Stage Booster Motor")

#plt.ylim([0,200])
#plt.xlim([0,8.7])
ax1.legend(loc=1)
plt.show()

In [ ]:
# write to file
with open("stage1_boost_thrust.csv", 'w') as out:
    for i, t in enumerate(stage1_boost_thrust):
        out.write("%0.3f, %0.1f\n" % (stage1_boost_time[i], t))

We can now use the booster data to create useful thrust curves for simulation.

 - [stage1_boost_thrust.csv](stage1_boost_thrust.csv)

Format:

 Column |                                Value | Units
 :----: | -----------------------------------: | :-----
  1     | Time                                 | s
  2     | Thrust                               | N
